# 一．AutoGraph使用规范

有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph。

TensorFlow 2.0主要使用的是动态计算图和Autograph。

动态计算图易于调试，编码效率较高，但执行效率偏低。

静态计算图执行效率很高，但较难调试。

而Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。

当然Autograph机制能够转换的代码并不是没有任何约束的，有一些编码规范需要遵循，否则可能会转换失败或者不符合预期。

## AutoGraph编码规范总结

- 1.被＠tf.function修饰的函数应尽可能使用TF中的函数而不是py中的其他函数，例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.
- 2.避免在@tf.function修饰的函数内部定义tf.Variable.
- 3.被@tf.function修饰的函数不可修改该函数外部的python列表或者字典等数据结构变量.

In [4]:
# 针对规范１
import tensorflow as tf
import numpy as np

@tf.function
def np_random():
    tf.print(np.random.randn(3,3))
@tf.function
def tf_random():
    tf.print(tf.random.normal((3,3)))

In [2]:
np_random()
np_random()
## 每次运行结果均相同

array([[-0.9280288 , -1.43434862,  1.56469118],
       [ 0.45911965,  1.24037181,  1.29833825],
       [-0.00991913,  1.73524171,  1.7599059 ]])
array([[-0.9280288 , -1.43434862,  1.56469118],
       [ 0.45911965,  1.24037181,  1.29833825],
       [-0.00991913,  1.73524171,  1.7599059 ]])


In [5]:
tf_random()
tf_random()

[[-0.595262408 0.312651902 0.372363687]
 [-0.374478161 -1.33422959 0.515737355]
 [-0.656585217 -0.716252506 2.11704707]]
[[0.245339885 -0.0484243333 -1.01997578]
 [-0.461177617 -1.37596643 -0.855163038]
 [-1.62632954 0.00532225613 0.672060788]]


In [6]:
## 针对规范２
x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def out_var():
    x.assign_add(1.0)
    tf.print(x)
    return x

out_var()

2


<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [7]:
@tf.function
def out_var():
    x = tf.Variable(1.0,dtype=tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return x

out_var()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ValueError: in converted code:

    <ipython-input-7-38cfc2f83669>:3 out_var  *
        x = tf.Variable(1.0,dtype=tf.float32)
    /home/pp/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /home/pp/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /home/pp/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /home/pp/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/def_function.py:502 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


In [8]:
# 针对规范３
tensor_list = []
def append_test():
    tensor_list.append(tf.constant(1))
    return tensor_list
append_test()
append_test()
print(tensor_list)

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=1>]


In [9]:
tensor_list = []
@tf.function
def append_test():
    tensor_list.append(tf.constant(1))
    return tensor_list
append_test()
append_test()
print(tensor_list)
## 结果不符合预期

[<tf.Tensor 'Const:0' shape=() dtype=int32>]


# 二．AutoGraph机制原理

当使用@tf.function装饰一个函数时，到底发生什么了呢？

例如以下代码：

In [26]:
##@tf.function(autograph=True)
@tf.function
def myadd(a,b):
    for i in tf.range(3):
        tf.print(i)
    c = a + b
    print('tracing')
    return c

后面什么都没有发生。仅仅是在Python堆栈中记录了这样一个函数的签名。

**当我们第一次调用这个被@tf.function装饰的函数时，后面到底发生了什么？**

例如我们写下如下代码。

In [27]:
myadd(tf.constant('hello'),tf.constant('world'))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

发生了2件事情，

第一件事情是创建计算图。

即创建一个静态计算图，跟踪执行一遍函数体中的Python代码，确定各个变量的Tensor类型，并根据执行顺序将算子添加到计算图中。 在这个过程中，如果开启了autograph=True(默认开启),会将Python控制流转换成TensorFlow图内控制流。 主要是将if语句转换成 tf.cond算子表达，将while和for循环语句转换成tf.while_loop算子表达，并在必要的时候添加 tf.control_dependencies指定执行顺序依赖关系。

相当于在 tensorflow1.0执行了类似下面的语句：
```python
g = tf.Graph()
with g.as_default():
    a = tf.placeholder(shape=[],dtype=tf.string)
    b = tf.placeholder(shape=[],dtype=tf.string)
    cond = lambda i: i<tf.constant(3)
    def body(i):
        tf.print(i)
        return(i+1)
    loop = tf.while_loop(cond,body,loop_vars=[0])
    loop
    with tf.control_dependencies(loop):
        c = tf.strings.join([a,b])
    print("tracing")
```
第二件事情是执行计算图。

相当于在 tensorflow1.0中执行了下面的语句：
```python
with tf.Session(graph=g) as sess:
    sess.run(c,feed_dict={a:tf.constant("hello"),b:tf.constant("world")})
```
因此我们先看到的是第一个步骤的结果：即Python调用标准输出流打印"tracing"语句。

然后看到第二个步骤的结果：TensorFlow调用标准输出流打印1,2,3。

**当我们再次用相同的输入参数类型调用这个被@tf.function装饰的函数时，后面到底发生了什么？**

例如我们写下如下代码。

In [24]:
myadd(tf.constant('hello'),tf.constant('world'))

0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

只会发生一件事情，那就是上面步骤的第二步，执行计算图。

所以这一次我们没有看到打印"tracing"的结果。

**当我们再次用不同的的输入参数类型调用这个被@tf.function装饰的函数时，后面到底发生了什么？**

例如我们写下如下代码。

In [25]:
myadd(tf.constant(1),tf.constant(2))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=int32, numpy=3>

由于输入参数的类型已经发生变化，已经创建的计算图不能够再次使用。

需要重新做2件事情：创建新的计算图、执行计算图。

所以我们又会先看到的是第一个步骤的结果：即Python调用标准输出流打印"tracing"语句。

然后再看到第二个步骤的结果：TensorFlow调用标准输出流打印1,2,3。

**需要注意的是，如果调用被@tf.function装饰的函数时输入的参数不是Tensor类型，则每次都会重新创建计算图。**

例如我们写下如下代码。两次都会重新创建计算图。因此，一般建议调用@tf.function时应传入Tensor类型。

In [21]:
myadd("hello","world")
myadd("good","morning")

tracing
0
1
2
tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'goodmorning'>

## 重新理解AutoGraph编码规范

1，被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print.

解释：Python中的函数仅仅会在跟踪执行函数以创建静态图的阶段使用，普通Python函数是无法嵌入到静态计算图中的，所以在计算图构建好之后再次调用的时候，这些Python函数并没有被计算，而TensorFlow中的函数则可以嵌入到计算图中。使用普通的Python函数会导致被@tf.function修饰前【eager执行】和被@tf.function修饰后【静态图执行】的输出不一致。

2，避免在@tf.function修饰的函数内部定义tf.Variable.

解释：如果函数内部定义了tf.Variable,那么在【eager执行】时，这种创建tf.Variable的行为在每次函数调用时候都会发生。但是在【静态图执行】时，这种创建tf.Variable的行为只会发生在第一步跟踪Python代码逻辑创建计算图时，这会导致被@tf.function修饰前【eager执行】和被@tf.function修饰后【静态图执行】的输出不一致。实际上，TensorFlow在这种情况下一般会报错。

3，被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

解释：静态计算图是被编译成C++代码在TensorFlow内核中执行的。Python中的列表和字典等数据结构变量是无法嵌入到计算图中，它们仅仅能够在创建计算图时被读取，在执行计算图时是无法修改Python中的列表或字典这样的数据结构变量的。

# 三．TF.model构建ＡutoGraph

前面在介绍Autograph的编码规范时提到构建Autograph时应该避免在@tf.function修饰的函数内部定义tf.Variable.

但是如果在函数外部定义tf.Variable的话，又会显得这个函数有外部变量依赖，封装不够完美。

一种简单的思路是定义一个类，并将相关的tf.Variable创建放在类的初始化方法中。而将函数的逻辑放在其他方法中。

惊喜的是，TensorFlow提供了一个基类tf.Module，通过继承它构建子类，我们可以非常方便地管理变量，还可以非常方便地管理它引用的其它Module，最重要的是，我们能够利用tf.saved_model保存模型并实现跨平台部署使用。

实际上，tf.keras.models.Model,tf.keras.layers.Layer 都是继承自tf.Module的，提供了方便的变量管理和所引用的子模块管理的功能。

因此，利用tf.Module提供的封装，再结合TensoFlow丰富的低阶API，实际上我们能够基于TensorFlow开发任意机器学习模型(而非仅仅是神经网络模型)，并实现跨平台部署使用。

## 1.应用tf.Model封装ＡutoGraph

In [2]:
# 定义一个简单的function
import tensorflow as tf
x = tf.Variable(1.,dtype=tf.float32)

## 在tf.function中input_signature限定输入张量的签名类型：shape和dtype
@tf.function(input_signature=[tf.TensorSpec(shape=[],dtype=tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return x

In [7]:
add_print(tf.constant(3.))
##add_print(tf.constant(1))　##输入了不符合张量签名的参数将报错

13


<tf.Tensor: shape=(), dtype=float32, numpy=13.0>

In [8]:
##继承tf.Module
class DemoModule(tf.Module):
    def __init__(self,init_value=tf.constant(0.0),name=None):
        super(DemoModule,self).__init__(name=None)
        with self.name_scope: ## 相当于with tf.name_scope('demo_module')
            self.x = tf.Variable(init_value,dtype=tf.float32,trainable=True)
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[],dtype=tf.float32)])
    def add_print(self,a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return self.x   

In [9]:
## execute
demo = DemoModule(init_value=tf.constant(1.))
demo.add_print(tf.constant(5.))

6


<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [15]:
## 查看全部变量和全部的可训练变量
demo.variables
demo.trainable_variables

(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)

(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)

In [16]:
## 查看全部的子模块
demo.submodules

()

In [17]:
## 使用tf.save_model保存模型，指定需要跨平台部署的方法
tf.saved_model.save(demo,'./../data/demo/11',signatures={'serving_default':demo.add_print})
##记载模型
demo2 = tf.saved_model.load('./../data/demo/11')
demo2.add_print(tf.constant(5.))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./../data/demo/11/assets
11


<tf.Tensor: shape=(), dtype=float32, numpy=11.0>

In [19]:
## 查看模型文件信息，具体信息在模型部署和跨平台使用时有可能会被用到
!saved_model_cli show --dir ./../data/demo/11 --all

2020-04-16 20:59:39.899616: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-04-16 20:59:39.899885: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvrtc.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2020-04-16 20:59:39.899899: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_I

In [24]:
## 在tensorboard中查看计算图，模块会被添加模块名demo_module，方便层次化呈现计算图结构
import datetime
logdir = './../data/demomodule/%s' %(datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
writer = tf.summary.create_file_writer(logdir)
#开启autograph跟踪
tf.summary.trace_on(graph=True,profiler=True)
##执行计算图
demo = DemoModule(tf.constant(1.))
result = demo.add_print(tf.constant(5.))

#将计算图信息写入日志
with writer.as_default():
    tf.summary.trace_export(name='demomodule',step=0,profiler_outdir=logdir)

## 启动tensorboard
%reload_ext tensorboard

6


In [25]:
from tensorboard import notebook
notebook.list()
notebook.start('--logdir ./../data/demomodule/')

No known TensorBoard instances running.


除了使用tf.Module的子类实现封装外，还可以通过tf.Module添加属性的方法进行封装

In [26]:
mymodule = tf.Module()
mymodule.x = tf.Variable(0.)

@tf.function(input_signature=[tf.TensorSpec(shape=[],dtype=tf.float32)])
def addprint(a):
    mymodule.x.assign_add(a)
    tf.print(mymodule.x)
    return mymodule.x
mymodule.addprint = addprint
mymodule.addprint(tf.constant(5.)).numpy()

5


5.0

In [27]:
mymodule.variables

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>,)

In [28]:
##保存和加载模型
tf.saved_model.save(mymodule,'./../data/mymodule/12',signatures={'serving_default':mymodule.addprint})

#加载模型
mymodule2 = tf.saved_model.load('./../data/mymodule/12')
mymodule2.addprint(tf.constant(1.))

INFO:tensorflow:Assets written to: ./../data/mymodule/12/assets
6


<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

### 3.2　tf.Module和tf.keras.Model，tf.keras.layers.Layer

以上两者军事继承自tf.Module实现，也具备变量管理和子模块的功能

In [29]:
from tensorflow.keras import models,layers,losses,metrics

In [34]:
issubclass(tf.keras.Model,tf.Module)
issubclass(tf.keras.layers.Layer,tf.Module)
issubclass(tf.keras.Model,tf.keras.layers.Layer)

True

True

True

In [35]:
tf.keras.backend.clear_session()

model = models.Sequential()

model.add(layers.Dense(4,input_shape=(10,)))
model.add(layers.Dense(2))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[-3.1340748e-01, -3.8998672e-01,  1.3250375e-01, -3.3326247e-01],
        [ 1.3608474e-01, -1.2577868e-01,  3.8215017e-01,  5.0758386e-01],
        [ 5.8723998e-01,  5.2090240e-01, -8.2731247e-05, -1.6712609e-01],
        [ 1.3883615e-01,  6.1983848e-01, -1.9427776e-02, -6.4031601e-01],
        [ 3.1576651e-01, -4.1325414e-01,  6.3392746e-01,  1.4729297e-01],
        [ 6.3313186e-01,  3.9338493e-01,  1.3482279e-01, -1.0084367e-01],
        [-1.9206464e-01,  5.7708144e-01, -4.9255809e-01,  1.2260878e-01],
        [-3.4013316e-01,  5.8758521e-01, -5.6420296e-01, -5.3258300e-01],
        [ 2.2392166e-01, -2.5935048e-01, -2.3307946e-01,  6.1826527e-01],
        [-2.7618998e-01,  5.6224561e-01, -6.4145261e-01, -1.6900063e-02]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy

In [37]:
## 冻结首层变量，使其变得不可训练
model.layers[0].trainable = False
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.34916615,  0.24203539],
        [-0.60658574,  0.98210263],
        [ 0.28745222,  0.24785686],
        [ 0.46875906, -0.64829874]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[-1.0603486],
        [-0.4425335]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [38]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7fcdb018a128>,
 <tensorflow.python.keras.layers.core.Dense at 0x7fcdb018a4a8>)

In [39]:
model.layers

In [41]:
model.name,model.name_scope()

('sequential', 'sequential')